In [7]:
from dotenv import load_dotenv
load_dotenv(r'.env')
import os
temp = os.environ.get('COMET_API_KEY')

In [6]:
# Importer comet_ml en haut de votre fichier, avant sklearn !
from comet_ml import Experiment

# Créer une expérience avec votre clé api
exp = Experiment(
    api_key=os.environ.get(temp),#ne pas coder en dur!
    project_name='milestone2',
    workspace='ift6758-a02',
)
# ... Faire de la science des données ...
exp.log_metrics({"auc": 12, "acc": 0.8, "loss": 0.65})


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/ift6758-a02/milestone2/9698eb944b8f4c95ae2ee66f31abf8db
COMET INFO:   Metrics:
COMET INFO:     acc  : 0.8
COMET INFO:     auc  : 12
COMET INFO:     loss : 0.65
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is l

COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information


## Ingénierie des caractéristiques I (10%)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Nettoyage des données

def clean_data(path) :
    """
    Nettoie un DataFrame de données de jeu NHL à partir d'un fichier JSON.

    Args:
        path (str): Le chemin vers le fichier JSON contenant les données de jeu NHL.

    Returns:
        pd.DataFrame: Un DataFrame propre contenant les colonnes 'emptyNet', 'isGoal',
                     'x', 'y', 'distanceToNet', et 'relativeAngleToNet'.

    """
    # Charger les données JSON en un DataFrame
    df = pd.read_json(path)

    # Extraire les informations des équipes, des événements de jeu et des résultats de jeu
    teamdf = df['gameData'].apply(pd.Series)['teams'].apply(pd.Series)
    df = df['liveData'].apply(pd.Series)['plays'].apply(pd.Series)['allPlays']
    df = pd.DataFrame(df)
    df = df.explode("allPlays")
    df = df.rename(columns={'allPlays': "play"})

    # Ajouter les noms des équipes 'awayTeam' et 'homeTeam'
    df['awayTeam'] = teamdf['away'].apply(pd.Series)['name']
    df['homeTeam'] = teamdf['home'].apply(pd.Series)['name']

    # Extraire les données d'événements 'play' pour obtenir les informations pertinentes
    df = df['play'].apply(pd.Series)
    df = pd.concat([df, df['result'].apply(pd.Series)], axis=1)

    # Filtrer les événements pour inclure uniquement 'Shot' et 'Goal'
    df = df[df['event'].isin(['Shot', 'Goal'])]

    # Extraire les coordonnées x et y des événements
    df['x'] = df['coordinates'].apply(pd.Series)['x']
    df['y'] = df['coordinates'].apply(pd.Series)['y']

    # Créer un nouveau DataFrame avec les colonnes pertinentes
    clean_df = df[['event',  'emptyNet','x','y']].copy()

    # Remplir les valeurs manquantes dans 'emptyNet' avec 0 et convertir en entier
    clean_df['emptyNet'] = clean_df['emptyNet'].fillna(0).astype(int)

    # Créer une nouvelle colonne 'isGoal' pour indiquer si l'événement est un but (1) ou non (0)
    clean_df['isGoal'] = (clean_df['event'] == 'Goal').astype(int)
    clean_df.drop('event',axis=1, inplace=True)

    # Coordonnées des camps gauche et droit
    coord_camp_gauche = (-90, 0)
    coord_camp_droit = (90, 0)

    # Calculer la distance du joueur au filet en utilisant les coordonnées x et y
    clean_df['distanceToNet'] = np.sqrt(np.minimum((clean_df['x'] - coord_camp_gauche[0])**2 + (clean_df['y'] - coord_camp_gauche[1])**2, (clean_df['x'] - coord_camp_droit[0])**2 + (clean_df['y'] - coord_camp_droit[1])**2))
    
    # Calculer l'angle relatif du joueur par rapport au filet (filet gauche)
    clean_df['relativeAngleToNet'] = np.degrees(np.arctan2(clean_df['y'], clean_df['x'] - coord_camp_gauche[0]))

    # Ajouter la colonne 'season' en extrayant l'année du chemin du fichier JSON
    clean_df['season'] = int(path[-9:-5])

    # Sauvegarder le DataFrame propre dans un fichier CSV
    clean_df.to_csv(path[:-5]+"_clean_M2.csv")


In [ ]:
# division des données en Ensemble de test et d'entrainement / validation

def split_data(path) :
    df = pd.read_csv(path)
    df.reset_index(drop=True, inplace=True)
    df = df.drop(columns=['Unnamed: 0.1'])
    
    train_data = df.loc[df['season'] != 2020]
    test_data = df.loc[df['season'] == 2020]

    test_data.reset_index(drop=True, inplace=True)

    train_data.to_csv("nhl_train_data.csv")
    test_data.to_csv("nhl_test_data.csv")

In [ ]:
# execution du code  (c'est long )
clean_data('nhl_data_2016.json') # ==> nouveau fichier nhl_data_2016_clean_M2.csv
clean_data('nhl_data_2017.json') # ==> nouveau fichier nhl_data_2017_clean_M2.csv
clean_data('nhl_data_2018.json') # ==> nouveau fichier nhl_data_2018_clean_M2.csv
clean_data('nhl_data_2019.json') # ==> nouveau fichier nhl_data_2019_clean_M2.csv
clean_data('nhl_data_2020.json') # ==> nouveau fichier nhl_data_2020_clean_M2.csv

p_clean = ["nhl_data_2016_clean_M2.csv","nhl_data_2017_clean_M2.csv", "nhl_data_2018_clean_M2.csv", "nhl_data_2019_clean_M2.csv", "nhl_data_2020_clean_M2.csv"]
df_list = []
count = 0
for path in p_clean:
    df = pd.read_csv(path)
    df['Unnamed: 0'] = df['Unnamed: 0']+count
    count = df['Unnamed: 0'].values[-1]
    df_list.append(df)
df = pd.concat(df_list, axis=0, ignore_index=True)
df.to_csv('nhl_all_years_clean_M2.csv')

In [ ]:
split_data('nhl_all_years_clean_M2.csv')